In [ ]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
#from bokeh.plotting import figure, show, output_file

In [ ]:
data = pd.read_csv(r'附件1 弱覆盖栅格数据(筛选).csv')
data.sort_values(by='traffic',inplace=True,ascending=False)
data = data.reset_index(drop=True)
data

In [ ]:
exsit=pd.read_csv(r'附件2 现网站址坐标(筛选).csv').drop('id',axis=1)
exsit

In [ ]:
#建立图坐标数据
coordinate=[]
coordinate=[[i,j]for i in range(2500)for j in range(2500)]
coordinate=pd.DataFrame(coordinate,columns=['x','y'])
coordinate

In [ ]:
#挑选半径为10内的点
def select_point10(x,y):
    point=[]
    for i in range(20):
        for j in range(20):
            if(math.sqrt((10-i)**2+(10-j)**2)<=10):
                point.append([x-10+i,y-10+j])
    point=pd.DataFrame(point,columns=['x','y'])
    return point

#挑选半径为30内的点
def select_point30(x,y):
    point=[]
    for i in range(60):
        for j in range(60):
            if(math.sqrt((30-i)**2+(30-j)**2)<=30):
                point.append([x-30+i,y-30+j])
    point=pd.DataFrame(point,columns=['x','y'])
    return point

#更新动态数据,即求DataFrame的差集
def renew(df1,df2):
    df1 = pd.concat([df1, df2, df2]).drop_duplicates(subset=['x','y'],keep=False)
    return df1

In [ ]:
#用现有基站坐标更新coordinate
for i in exsit.index:
    tmp=select_point10(exsit.x[i],exsit.y[i])
    coordinate=renew(coordinate,tmp)
coordinate

In [ ]:
#初始化接收数组和业务量总和
bs=[]
traffic_sum=0

In [ ]:
#遍历弱覆盖栅格数据（排序）
while(1):
    
    point10=select_point10(data.x.iloc[0],data.y.iloc[0])
    point10=pd.merge(coordinate, point10, how='inner')
    
    point30=select_point30(data.x.iloc[0],data.y.iloc[0])
    point30=pd.merge(coordinate, point30, how='inner')
    
    #遍历可建基站点
    ratio10=[0]
    for temp1 in point10.index:
        point10_10=select_point10(point10.x[temp1],point10.y[temp1])
        point10_10=pd.merge(data, point10_10, how='inner')
        if ratio10[-1]<point10_10.traffic.sum():
            ratio10.append(point10_10.traffic.sum())
            x_max_10=point10_10.x[0]
            y_max_10=point10_10.y[0]
        
    ratio30=[0]
    for temp1 in point30.index:
        point30_30=select_point30(point30.x[temp1],point30.y[temp1])
        point30_30=pd.merge(data, point30_30, how='inner')
        if ratio30[-1]<point30_30.traffic.sum():
            ratio30.append(point30_30.traffic.sum()/10)
            x_max_30=point30_30.x[0]
            y_max_30=point30_30.y[0]
    
    #建立新的基站    
    if ratio10[-1]>ratio30[-1]:
        traffic_sum=traffic_sum+ratio10[-1]
        x_max=x_max_10
        y_max=y_max_10
        point_new=select_point10(x_max,y_max)
        bs.append([x_max,y_max,10])
        
    if ratio10[-1]<ratio30[-1]:
        traffic_sum=traffic_sum+ratio30[-1]
        x_max=x_max_30
        y_max=y_max_30
        point_new=select_point10(x_max,y_max)
        bs.append([x_max,y_max,30])
        
    if ratio10[-1]== ratio30[-1]:
        data=data.iloc[1:,:]

    #更新动态数据
    ttt=select_point10(x_max,y_max)
    coordinate=renew(coordinate,ttt)
    data=renew(data,point_new)
    
    #在业务量大于总业务量90%时结束循环
    if traffic_sum>=6350607.1:
        break

        
    print(traffic_sum)

In [ ]:
#保存数结果据
data.to_csv("data最终结果.csv")
pd.DataFrame(bs,columns=['x','y','r']).to_csv("bs最终结果.csv")
coordinate.to_csv("coordinate最终结果.csv")